# The A-Z of Python Dictionaries

Python dictionaries are great. Once you learn how to use them, its clear that they are a versatile, easy-to-use, efficient data strucutre. But its not just covenience. Dictioanries are core to how Python works under the hood. Module namespaces, class and inheritance attributes, and function keywork arguments all live inside dictionaries. Understanding how you can leveraging dictionaries to their maximum potential is essential in becoming a Pythonista.

There might a few things you don't know about Python dictionaries, so lets see if I can teach you something. 

I will first cover a few tricks to take your dictionary game to the next level, then I will dive into the juicy CPython implementation of dictionaries, so we can understand the practical consequences of how they actually work.

I'll assume you know the basics. If you don't, check out the great tutorals at [Tutorials Point](https://www.tutorialspoint.com/python/python_dictionary.htm) or [Sentdex](https://www.youtube.com/watch?v=YNRc6c0wUA8). I'll help take that knowledge to the next level. The basic idea is that this data structure with keys and values, where each key is a unique string, and the values can be any object. New to Python 3.6, dictionaries keep their ordering from creation. 

In [3]:
a = {'key1': 1,
     'key2': 'hi',
     'key3': None}
a

{'key1': 1, 'key2': 'hi', 'key3': None}

In [5]:
a['key2'] = 3.14
a

{'key1': 1, 'key2': 3.14, 'key3': None}

Bear in mind that like lists, the contents of a dictionary's values can be of any type.

### Merging Dictionaries

Python 3.5 was a solid update. It allows us to merge dictionaries using a technique called function argument unpacking, which I cover in [another post here](function_argument_unpacking.ipynb). By using the `**` operator, we can unpack a dictionaries contents, and use that as arugements to the constructor of another dictionary object:

In [7]:
b = {'key4': 1234}
c = {**a, **b}
c

{'key1': 1, 'key2': 3.14, 'key3': None, 'key4': 1234}

### Function Argument Unpacking

This idea of function argument unpacking can extend to, well, functions. You can unpack the contents of a dictionary as arguments to a function. If a single `*` operator is used, then thekeys get used as arugments, but if you use `**`, then each key will try and find its matching argument name. But be careful that all the dictionary keys match up with the function argument names, or you will get a `Key Error`.

In [10]:
d = {'x': 1,
     'y': 2,
     'z': 3}

def func(x, y, z):
    print(x, y, z)
    
func(*d)

x y z


In [11]:
func(**d)

1 2 3


### Dictionary Comprehnsions

In addition to the `dict` constructor, we can use dictionary comprehensions to instantiate dictionaries. Just like list comprehensions, where we have a compact and efficient way of create a list, we can use the same technique with dictionaries, just substituting the square brackets for curly ones. The format is 

```python
{key: value for item in iterator}
```

In [20]:
squares = {str(i): i**2 for i in range(10)}
a

{'0': 0,
 '1': 1,
 '2': 4,
 '3': 9,
 '4': 16,
 '5': 25,
 '6': 36,
 '7': 49,
 '8': 64,
 '9': 81}

### Default Values for Dictionaries

There are two ways to retreive values from dictionaries. The first is the most common, and most people know this: the method of using square brackets and the string of the key you are searching for:

In [25]:
squares['5']

25

But you can also use the `get` method to access the value the same way. This gives an added benefit.Specifically, we can set a default return value if the key we are searching for is not present in the dictionary. This has a great use case in avoiding unwanted errors, or supplying a baseline value if a the one you are searching for is not yet populated.

In [26]:
squares.get('100', 'not found')

'not found'

### The Switch Case for dicts

Python sadly does not ship with the switch case statement which many other languages enjoy. The switch case provides a more concise interface for if/else statements. It might look something like this: 

```c
switch(val){
    case 1
    do something
    break
    
    case2
    do something else
    break
    
    case 3
    do something else
    break
    ...
}
```

We can slightly emulate this functionality in Python by leveraging the fact that functions are objects. We can create as many functions as we have cases, and each function will return whatever we need it to. Then we create a dictionary with keys (conditions) and values (functions to do something if condition is met). Now instead of evaulating for a condition like we would an if/else, we instead use the would-be condition as a key. If the key matches the one in the dictionary, we return the corresponding function and call it. You might have to get a little bit hacky to solve your specific problem, but hey, its a work around!  

In [29]:
def case1(): return 1
def case2(): return 2
def case3(): return 3

switch = {
    'a': case1,
    'b': case2,
    'c': case3,
}

switch['a']()

1

In [30]:
switch['b']()

2

## How Python Dictionaries Work Under the Hood

### Hashes

Alright, now the juicy stuff, a 30000ft overview of the CPython `dict` iternals, but more importantly, its practical consequences.

How does a dictionary work under the hood? Well first we have to understand Python's built in `hash` function. What is a hash? A hash is a unique pattern of characters that are created when a hashing algorithm is applied to an object. For an object to be hashable, it has to have a few properties:

1. it must not change over the lifetime of the program. This means things like tuples, integers etc. A mutable object like a list is not hashable for this reason. 

2. it must have a `__eq__` method, which means that we can compare it to other things for equality. If those two things are equal, they must have the same hash. 

3. if `a==b` is `True`, then `hash(a) == hash(b)` is also `True`

Below we try a few examples. 

Note that `1` and `1.0` have the same hash, even though one is a float and one is an integer (they are still equal though). 

In [34]:
# tuples are immutable, so hashable
hash((0, 1, 2))

1267305975155491464

In [42]:
# equivalency means the same hash
print(hash(13523))
print(hash(13523.0))

13523
13523


In [39]:
# semantic similarity ideally leads to very different hashes
print(hash('Mary'))
print(hash('Marie'))

-3002527284114270182
5208799245293544286


In [41]:
# no mutable objects
hash([0, 1, 2])

TypeError: unhashable type: 'list'

### How it Works

Well, now that hashes are out of the way, you might not be surprised to learn that Python dictionaries and sets use hashes (in the form of hash tables). A hash table is a sparse array (an array with empty cells). Each cell of a hash table is sometimes called a "bucket", so we'll use that terminology. For a `dict`, there is a bucket for each item, with two fields: a reference to the key, and a reference to the value. Note: this is a reference pointer, not the actual object! This is important, because no matter what object that referece is pointing to, the reference itself has the same size, so we can index the `dict` hashtable using offsets. 

<img src="files/hashmap.png" width="400" alt="https://en.wikipedia.org/wiki/Hash_table">

In general, the Python interpreter will try to keep 1/3 of the buckets empty, and copy over the data to a larger `dict` if the preallocated space fills up. The reason for the sparsity is becasue scattering around the hash indices makes retrieval faster, as we will see. Ideally, keys with similar but different semantic values (ex. 'Mary' vs. 'Marie') will create very different hashses, and hashes which *are* similar will be far away from each other. 

So how does retrieval actually work? Well if we call `d['key']`, where `d` is a `dict`, then a hash is computed of `key`, and the "least significant" part of that hash is used to locate a bucket in the hash table. The beginning of the hash is the most significant, and the end is the least, so here maybe the last 3 characters were used. Only part of the hash is used so that there is a smaller field of numbers to compare for a match. If nothing is found, a `KeyError` is raised. If there is a match, then the found value is returned. 

However, you may have spotted an error here. Only **part** of the hash is used. And since the hash function maps an arbitrary number of objects to a finite number of hash values, that means we can, and very often do, get conflicts where two partial hashes match in a given table, but the full hashes do not match. This is called a hash collision, and its not good. Its why the hash table is a sparse array, to try and avoid these conflicts. If a hash conflict does occur, then a different part of the hash is used for comparisons, and this is repeated until a found value is returned, or a `KeyError` is raised. 

<img src="files/dict_flowchart.jpg" width="600" alt="Fluent Python by Luciano Ramalho">

So to recap, using the partial hash is a way to make this retrieval faster, at the cost of collisions. Collisions are kept to a minimum by using a sparse hash table. If a collision occurs, a different part of the hash is used for retrieval. This process is repeated till a sucessful retrieval or a failure. 

The process for updating a dictionary is the same, except when an empty bucket is found, the new value is put there, instead of a `KeyError`. And when a bucket is found as an exact match, it is overwritten with the new value. Additionally, when inserting new values, if the 1/3 threshold of sparsity is exceeded, then the hash table is rebuilt to be bigger, and a larger part of the hash is used for the partial-indexing. This keeps the rate of collisions low. 


### Practical Consequences: Space, Time, and Sparsity

While this sounds more complicated that necessary, in reality, collisions are quite low, with between 0-2 collisions per search on average. The true benefit is that Understanding how `dict` works gives some insights in how to properly use it:

1. Space inefficency 
2. Time effiency
3. Order impermanance

The first insight is that dictionaries are not memory efficient. Since the are 2/3 sparse, the uses 66% more space then the actual data we use them for. If you have a large list of records, a list of tuples or named tuples might be the best data structure, since you can avoid the overhead of hash tables and using the name field. 

But, the high cost of space is a trade off for the low cost of speed. As long as your `dict` fits into memory, it will have fast access to the values. 

As a dictionary outgrows itself, it needs to get resized to maintain its sparsity. The practical consequences of this are that order is not retained before and after resizing. This means that iterating over a dictionary and adding or deleting items can be dangerous! 

```python
#dont do this!
for k, v in d.items():
    if v < 10:
        del d[k] 
        
Traceback (most recent call last):
File "<stdin>", line  1,in module
RuntimeError: dictionary changed size during iteration
```

### Sets

It should be noted that `sets` use the same hash table infrastructure as `dict`s, except the only hold one reference, the value, and have no keys. Sets are great for creating an array of values where each value is unique, and there are no duplicates (since hash collisions would prevent this).  

**--------------------------------------------**
## Resources
- Image source: Wikipedia https://en.wikipedia.org/wiki/Hash_table

**Books**

- [Python Tricks by Dan Bader](https://www.amazon.com/Python-Tricks-Buffet-Awesome-Features/dp/1775093301)
- [Fluent Python by Luciano Ramalho](https://www.amazon.com/Fluent-Python-Concise-Effective-Programming/dp/1491946008/ref=sr_1_1?s=books&ie=UTF8&qid=1534450496&sr=1-1&keywords=fluent+python)
- [Slides on Python Dicts](https://www.fusionbox.com/media/filer_public/c9/a3/c9a3bfce-b706-45e7-878e-7bc0d273c089/talk.pdf)